In [65]:
import pandas as pd
import numpy as np

In [ ]:
# Ler o arquivo CSV
df= pd.read_csv('merged.csv', sep=',')

In [67]:
# Visualizar as primeiras linhas do DataFrame
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C1D650626C8C899A,electric_bike,2024-01-12 15:30:27,2024-01-12 15:37:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.903267,-87.634737,41.889177,-87.638506,member
1,EECD38BDB25BFCB0,electric_bike,2024-01-08 15:45:46,2024-01-08 15:52:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902937,-87.634440,41.889177,-87.638506,member
2,F4A9CE78061F17F7,electric_bike,2024-01-27 12:27:19,2024-01-27 12:35:19,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902951,-87.634470,41.889177,-87.638506,member
3,0A0D9E15EE50B171,classic_bike,2024-01-29 16:26:17,2024-01-29 16:56:06,Wells St & Randolph St,TA1305000030,Larrabee St & Webster Ave,13193,41.884295,-87.633963,41.921822,-87.644140,member
4,33FFC9805E3EFF9A,classic_bike,2024-01-31 05:43:23,2024-01-31 06:09:35,Lincoln Ave & Waveland Ave,13253,Kingsbury St & Kinzie St,KA1503000043,41.948797,-87.675278,41.889177,-87.638506,member


In [68]:
# Resumo inicial dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5860568 entries, 0 to 5860567
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 581.3+ MB


In [69]:
df.describe()

,start_lat,start_lng,end_lat,end_lng
count,5.860568e+06,5.860568e+06,5.853336e+06,5.853336e+06
mean,4.190218e+01,-8.764622e+01,4.190258e+01,-8.764644e+01
std,4.474907e-02,2.748958e-02,5.598336e-02,1.136824e-01
min,4.164000e+01,-8.791000e+01,1.606000e+01,-1.440500e+02
25%,4.188096e+01,-8.766000e+01,4.188096e+01,-8.766000e+01
50%,4.189736e+01,-8.764182e+01,4.189776e+01,-8.764288e+01
75%,4.193000e+01,-8.762952e+01,4.193000e+01,-8.762954e+01
max,4.207000e+01,-8.752000e+01,8.796000e+01,1.525300e+02


### 1. Tratar Valores Ausentes

In [70]:
# Mostrar os valores ausentes por coluna em porcentagem
missing_values = (df.isnull().mean() * 100).round(2)
missing_values

ride_id                0.00
rideable_type          0.00
started_at             0.00
ended_at               0.00
start_station_name    18.33
start_station_id      18.33
end_station_name      18.85
end_station_id        18.85
start_lat              0.00
start_lng              0.00
end_lat                0.12
end_lng                0.12
member_casual          0.00
dtype: float64

In [71]:
# Preenchendo estaçoes com valores ausentes com Unknown
df['start_station_name'] = df['start_station_name'].fillna('Unknown')
df['end_station_name'] = df['end_station_name'].fillna('Unknown')

### 2. Verificar e Remover Duplicatas

In [72]:
# Remover linhas com coordenadas ausentes
df = df.dropna(subset=['start_lat', 'start_lng', 'end_lat', 'end_lng'])

In [73]:
# Verificar duplicatas em ride_id
duplicates = df.duplicated(subset=['ride_id']).sum()
print(f"Número de duplicatas em ride_id: {duplicates}")

Número de duplicatas em ride_id: 171


In [74]:
# Removendo duplicatas, mantendo a primeira ocorrência
df = df.drop_duplicates(subset=['ride_id'], keep='first')

### 3. Corrigir Tipos de Dados

In [75]:
# Remove milissegundos com regex (ex: ".289" de "2021-08-01 12:34:56.289")
df['started_at'] = df['started_at'].str.replace(r'\.\d+', '', regex=True)
df['ended_at'] = df['ended_at'].str.replace(r'\.\d+', '', regex=True)

# Verificar se houve a remoção de milissegundos
print(df['started_at'].str.contains(r'\.\d+').any()) # Retorna False se não houver milissegundos e True se houver

False


In [76]:
# Agora converte com segurança
df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S')
df['ended_at'] = pd.to_datetime(df['ended_at'], format='%Y-%m-%d %H:%M:%S')

In [77]:
# Converter member_casual para categoria
df['member_casual'] = df['member_casual'].astype('category')

In [78]:
# Verificar o tipos dos dados
df.dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual               category
dtype: object

In [79]:
# Verifica quantas linhas tem valores NaT (Not a Time) após a conversão
print("Started_at NaT:", df['started_at'].isna().sum())
print("Ended_at NaT:", df['ended_at'].isna().sum())


Started_at NaT: 0
Ended_at NaT: 0


### 4. Tratar Durações de Viagem Inválidas

-- A duração da viagem (ended_at - started_at) deve ser positiva e razoável. Viagens com duração negativa ou excessivamente longa (ex.: >24 horas) podem ser erros.

In [80]:
# Calcula a duração da viagem em minutos
df['duration_minutes'] = ((df['ended_at'] - df['started_at']).dt.total_seconds() / 60).round(2)


Identificar Viagens com Coordenadas Idênticas

In [81]:
# Identificar viagens com coordenadas idênticas
df['same_location'] = (abs(df['start_lat'] - df['end_lat']) < 1e-6) & (abs(df['start_lng'] - df['end_lng']) < 1e-6)

# Contar quantas viagens têm coordenadas idênticas
same_location_count = df['same_location'].sum()
print(f"Viagens com coordenadas idênticas (não saiu do lugar): {same_location_count}")

# Opcional: Visualizar algumas dessas viagens
print(df[df['same_location']][['start_lat', 'start_lng', 'end_lat', 'end_lng', 'duration_minutes', 'member_casual']].head())

Viagens com coordenadas idênticas (não saiu do lugar): 333797
      start_lat  start_lng    end_lat    end_lng  duration_minutes  \
3173  41.872229 -87.661364  41.872229 -87.661364              0.55   
3176  41.872229 -87.661364  41.872229 -87.661364              2.17   
3178  41.929143 -87.649077  41.929143 -87.649077              5.67   
3179  41.929143 -87.649077  41.929143 -87.649077              0.32   
3182  41.899643 -87.667700  41.899643 -87.667700              0.47   

     member_casual  
3173        member  
3176        member  
3178        member  
3179        member  
3182        member  


Identificar Viagens com Duração Muito Curta

In [82]:
# Definir limite para duração muito curta (em minutos)
min_duration = 5

# Identificar viagens com duração < 1 minuto
short_duration = df['duration_minutes'] < min_duration
short_duration_count = short_duration.sum()
print(f"Viagens com duração < {min_duration} minuto: {short_duration_count}")

# Opcional: Visualizar algumas dessas viagens
print(df[short_duration][['duration_minutes', 'start_lat', 'end_lat', 'member_casual']].head())

Viagens com duração < 5 minuto: 1240558
    duration_minutes  start_lat    end_lat member_casual
15              3.70  41.884175  41.889177        member
21              2.08  41.888243  41.889177        member
22              3.35  41.888243  41.889177        member
30              2.48  41.920082  41.921057        member
43              2.25  41.875933  41.877708        member


In [83]:
# Remove viagens com duração negativa, menores que 5 minutos ou maiores que 24 horas
df = df[
    (df['duration_minutes'] >= 5) &
    (df['duration_minutes'] <= 1440)
]

In [84]:
# Remove viagens onde a localização inicial e final são iguais
df = df[
    ~((df['start_lat'] == df['end_lat']) & (df['start_lng'] == df['end_lng']))
]

In [85]:
# Visualiza distribuição da duração das viagens
print(df['duration_minutes'].describe())

count    4.445349e+06
mean     1.773484e+01
std      3.017693e+01
min      5.000000e+00
25%      7.950000e+00
50%      1.195000e+01
75%      1.950000e+01
max      1.439820e+03
Name: duration_minutes, dtype: float64


### 6. Corrigir Inconsistências em **rideable_type** e **member_casual**
-- Valores inesperados em rideable_type (ex.: tipos de bicicletas não reconhecidos) ou member_casual (ex.: valores além de "casual" e "member") podem indicar erros.

In [86]:
# Verificar valores unicos
print(df['rideable_type'].unique())
print(df['member_casual'].unique())

# Nao temos problemas com valores unicos

['electric_bike' 'classic_bike' 'electric_scooter']
['member', 'casual']
Categories (2, object): ['casual', 'member']


### 7. Tratar Inconsistências entre Estações e IDs
As colunas **start_station_name** e **start_station_id** (e suas contrapartes **end_**) devem ser consistentes. Um mesmo station_id deve corresponder sempre ao mesmo station_name.

In [87]:
# Verifica consistencia entre station_id e station_name
start_station_check = df.groupby('start_station_id')['start_station_name'].nunique()

inconsistent_stations = start_station_check[start_station_check > 1]
print(f"Estações de início com IDs inconsistentes: {inconsistent_stations}")

Estações de início com IDs inconsistentes: start_station_id
1524189         2
20129           2
21322           2
21366           2
21371           2
               ..
661             2
662             2
KA1503000074    2
KA17018054      2
TA1305000030    2
Name: start_station_name, Length: 76, dtype: int64


In [88]:
# Se houver inconsistências, corrigir (ex.: manter o nome mais comum para cada ID)
if not inconsistent_stations.empty:
    # Mapear ID para o nome mais frequente
    station_mapping= df.groupby('start_station_id')['start_station_name'].agg(lambda x: x.mode()[0]).to_dict()

    # Aplicar o mapeamento
    df['start_station_name'] = df['start_station_id'].map(station_mapping).fillna(df['start_station_name'])

In [89]:
# Repetir para end_station_id
end_station_check = df.groupby('end_station_id')['end_station_name'].nunique()
inconsistent_end = end_station_check[end_station_check > 1]
print(f"Estações de fim com IDs inconsistentes: {inconsistent_end}")

Estações de fim com IDs inconsistentes: end_station_id
1524189                                2
20129                                  2
21322                                  2
21366                                  2
21371                                  2
                                      ..
665                                    2
Hubbard Bike-checking (LBS-WH-TEST)    3
KA1503000074                           2
KA17018054                             2
TA1305000030                           2
Name: end_station_name, Length: 83, dtype: int64


In [90]:
# Se houver inconsistências, corrigir (ex.: manter o nome mais comum para cada ID)

if not inconsistent_end.empty:
    # Mapear ID para o nome mais frequente
    end_station_mapping = df.groupby('end_station_id')['end_station_name'].agg(lambda x: x.mode()[0]).to_dict()

    # Aplicar o mapeamento
    df['end_station_name'] = df['end_station_id'].map(end_station_mapping).fillna(df['end_station_name'])

In [91]:
df.sample(100)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_minutes,same_location
2633843,3AD0FC642418E360,electric_bike,2024-07-31 14:57:16,2024-07-31 15:08:24,Field Museum,13029,Federal St & Polk St,SL-008,41.865312,-87.617867,41.872078,-87.629544,casual,11.13,False
5098916,08AE13A624BDA846,electric_bike,2024-10-16 05:49:37,2024-10-16 05:56:11,Sheridan Rd & Buena Ave,TA1309000027,Broadway & Barry Ave,13137,41.958494,-87.654966,41.937582,-87.644098,member,6.57,False
1203184,0B5DC59600013883,classic_bike,2024-05-21 17:22:02,2024-05-21 17:31:26,Kingsbury St & Kinzie St,KA1503000043,Canal St & Taylor St,15550,41.889177,-87.638506,41.870257,-87.639474,member,9.40,False
534480,B5539D26133CEC4B,classic_bike,2024-03-14 16:40:32,2024-03-14 16:57:39,Sheffield Ave & Willow St,TA1306000032,Burling St & Diversey Pkwy,TA1309000036,41.913688,-87.652855,41.933140,-87.647760,member,17.12,False
2912479,C22860D205BF650F,electric_bike,2024-07-18 13:30:00,2024-07-18 13:47:50,Leavitt St & Armitage Ave,TA1309000029,Michigan Ave & Oak St,13042,41.917805,-87.682437,41.900960,-87.623777,member,17.83,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992775,D13948D62E0376E3,classic_bike,2024-06-04 16:07:56,2024-06-04 16:26:09,Daley Center Plaza,TA1306000010,Loomis St & Lexington St,13332,41.884241,-87.629634,41.872229,-87.661364,member,18.22,False
4133490,4D4F9C445A6A6D3F,electric_scooter,2024-09-11 15:24:22,2024-09-11 15:49:56,Unknown,NaN,Southport Ave & Wrightwood Ave,TA1307000113,41.890000,-87.620000,41.928773,-87.663913,member,25.57,False
2631565,1E9832A232BDA8AE,electric_bike,2024-07-24 11:05:07,2024-07-24 11:19:04,LaSalle Dr & Huron St,KP1705001026,DuSable Lake Shore Dr & Diversey Pkwy,TA1309000039,41.894678,-87.632137,41.932588,-87.636427,member,13.95,False
723503,08FD25EA85A4A2AC,electric_bike,2024-04-19 17:13:57,2024-04-19 17:21:02,N Sheffield Ave & W Wellington Ave,20256.0,Clark St & Grace St,TA1307000127,41.936343,-87.653793,41.950780,-87.659172,member,7.08,False


### 8. Criar coluna **day_of_week** e **month**

In [95]:
# Criar a coluna dia da semana
df['day_of_week'] = df['started_at'].dt.day_name() # 0=Monday, 1=Tuesday, ..., 6=Sunday

df['day_of_week']

0             Friday
1             Monday
2           Saturday
3             Monday
4          Wednesday
             ...    
5860563    Wednesday
5860564       Monday
5860565      Tuesday
5860566       Sunday
5860567      Tuesday
Name: day_of_week, Length: 4445349, dtype: object

In [97]:
# Criar coluna de month
df['month'] = df['started_at'].dt.month_name() # 1=January, 2=February, ..., 12=December
df['month']

0           January
1           January
2           January
3           January
4           January
             ...   
5860563    December
5860564    December
5860565    December
5860566    December
5860567    December
Name: month, Length: 4445349, dtype: object

In [98]:
df.head(100)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_minutes,same_location,day_of_week,month
0,C1D650626C8C899A,electric_bike,2024-01-12 15:30:27,2024-01-12 15:37:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.903267,-87.634737,41.889177,-87.638506,member,7.53,False,Friday,January
1,EECD38BDB25BFCB0,electric_bike,2024-01-08 15:45:46,2024-01-08 15:52:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902937,-87.634440,41.889177,-87.638506,member,7.22,False,Monday,January
2,F4A9CE78061F17F7,electric_bike,2024-01-27 12:27:19,2024-01-27 12:35:19,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902951,-87.634470,41.889177,-87.638506,member,8.00,False,Saturday,January
3,0A0D9E15EE50B171,classic_bike,2024-01-29 16:26:17,2024-01-29 16:56:06,Wells St & Randolph St,TA1305000030,Larrabee St & Webster Ave,13193,41.884295,-87.633963,41.921822,-87.644140,member,29.82,False,Monday,January
4,33FFC9805E3EFF9A,classic_bike,2024-01-31 05:43:23,2024-01-31 06:09:35,Lincoln Ave & Waveland Ave,13253,Kingsbury St & Kinzie St,KA1503000043,41.948797,-87.675278,41.889177,-87.638506,member,26.20,False,Wednesday,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,2DE16C49B601A61B,electric_bike,2024-01-04 12:24:11,2024-01-04 12:45:09,Francisco Ave & Chicago Ave,411,Franklin St & Jackson Blvd,TA1305000025,41.895773,-87.698624,41.877708,-87.635321,member,20.97,False,Thursday,January
111,8B07F83B63CE1EDB,classic_bike,2024-01-05 18:46:06,2024-01-05 18:56:33,Delano Ct & Roosevelt Rd,KA1706005007,Halsted St & 18th St,13099,41.867491,-87.632190,41.857506,-87.645991,member,10.45,False,Friday,January
112,42A5AFA44BE3C52F,electric_bike,2024-01-23 16:51:52,2024-01-23 17:02:07,Canal St & Madison St,13341,Ogden Ave & Race Ave,13194,41.882412,-87.639582,41.891795,-87.658751,member,10.25,False,Tuesday,January
113,DFB0DA65074A1263,classic_bike,2024-01-05 17:07:52,2024-01-05 17:24:03,Field Museum,13029,Michigan Ave & Ida B Wells Dr,TA1305000010,41.865312,-87.617867,41.876243,-87.624426,casual,16.18,False,Friday,January


In [99]:
# Salvar o DataFrame limpo em um novo arquivo CSV
df.to_csv('cyclistic_cleaned.csv', index=False)